In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "usdjpy"
MINUTES = 15
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

# Build base name
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
NORMALIZED_DIR = BASE_DIR / "normalized"
SCALER_DIR = BASE_DIR / "scalers"

# Final paths
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_processed.pkl"
NORMALIZED_FILE_PATH = NORMALIZED_DIR / f"{RESAMPLED_NAME}_normalized.pkl"
STD_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_standard_scaler.pkl"
MINMAX_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_minmax_scaler.pkl"

In [3]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,timestamp,open,high,low,close,volume,spread,log_volume,close_delta,close_return,...,dom_sin,dom_cos,month,month_sin,month_cos,label,train_label,3b_label,3b_train_label,ttrain_label
0,2020-01-02 12:45:00+00:00,108.789,108.7955,108.7635,108.7870,1.398160e+06,0.001986,14.150668,-0.0010,-0.000009,...,0.394356,0.918958,1,0.5,0.866025,0,0,-1,0,1
1,2020-01-02 13:00:00+00:00,108.787,108.7900,108.7720,108.7780,1.224630e+06,0.001864,14.018150,-0.0090,-0.000083,...,0.394356,0.918958,1,0.5,0.866025,0,0,-1,0,1
2,2020-01-02 13:15:00+00:00,108.777,108.7770,108.7090,108.7270,2.424340e+06,0.001921,14.701070,-0.0510,-0.000469,...,0.394356,0.918958,1,0.5,0.866025,-1,0,-1,0,0
3,2020-01-02 13:30:00+00:00,108.726,108.7390,108.6570,108.6945,2.223350e+06,0.002018,14.614526,-0.0325,-0.000299,...,0.394356,0.918958,1,0.5,0.866025,-1,1,-1,0,0
4,2020-01-02 13:45:00+00:00,108.694,108.7245,108.6885,108.6940,6.457450e+06,0.002147,15.680745,-0.0005,-0.000005,...,0.394356,0.918958,1,0.5,0.866025,0,0,-1,0,1


In [4]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,dom_sin,dom_cos,month,month_sin,month_cos,label,train_label,3b_label,3b_train_label,ttrain_label
count,123945.000000,123945.000000,123945.000000,123945.000000,1.239450e+05,123945.000000,123945.000000,123945.000000,123945.000000,123945.000000,...,1.239450e+05,123945.000000,123945.000000,1.239450e+05,1.239450e+05,123945.000000,123945.000000,123945.000000,123945.000000,123945.000000
mean,127.990416,128.035766,127.943652,127.990860,6.337400e+06,0.006639,15.182260,0.000394,0.000003,0.000003,...,2.353235e-03,-0.022177,6.521973,-4.503608e-03,-4.841234e-03,0.017354,1.018484,0.023938,1.023938,1.017354
std,18.398600,18.410248,18.385790,18.398718,6.365523e+06,0.009127,1.066178,0.081064,0.000601,0.000602,...,7.139405e-01,0.699857,3.439884,7.071395e-01,7.070488e-01,0.795467,0.823626,0.797676,0.797676,0.795467
min,101.349000,101.954000,101.182000,101.349000,0.000000e+00,0.001347,0.000000,-3.885500,-0.028329,-0.028738,...,-9.987165e-01,-0.994869,1.000000,-1.000000e+00,-1.000000e+00,-1.000000,0.000000,-1.000000,0.000000,0.000000
25%,109.436000,109.460500,109.410500,109.435500,1.971010e+06,0.003175,14.494057,-0.027000,-0.000217,-0.000217,...,-7.247928e-01,-0.758758,4.000000,-8.660254e-01,-8.660254e-01,-1.000000,0.000000,-1.000000,0.000000,0.000000
50%,130.790500,130.857000,130.725500,130.790500,4.233180e+06,0.005348,15.258464,0.001000,0.000007,0.000007,...,-2.449294e-16,-0.050649,7.000000,-2.449294e-16,-1.836970e-16,0.000000,1.000000,0.000000,1.000000,1.000000
75%,145.526000,145.580500,145.471000,145.527000,8.654850e+06,0.006976,15.973631,0.029500,0.000232,0.000232,...,7.247928e-01,0.688967,10.000000,5.000000e-01,5.000000e-01,1.000000,2.000000,1.000000,2.000000,2.000000
max,161.941000,161.951000,161.917500,161.940000,2.393905e+08,0.276667,19.293607,2.052000,0.014463,0.014360,...,9.987165e-01,1.000000,12.000000,1.000000e+00,1.000000e+00,1.000000,2.000000,1.000000,2.000000,2.000000


In [5]:
FEATURES_COLS = [
    # Basic Data
    'close_log_return',
    'log_volume',
    'spread',

    # Other
    'ret_mean_5',
    'ret_mean_10'
]
TIME_FEATURE_COLS = [
    'hour_cos',
    'dow_cos',
    'dom_cos',
    'month_cos',
]
TA_MIN_MAX = [
    # TA
    'rsi_14',
]
TA_STD = [
    'ema_21',
    'sma_50',
    'atr_14',
    
    'bb_upper',
    'bb_lower',
    'bb_mavg',
    'bb_width',

    'donchian_upper',
    'donchian_lower',
    'donchian_mid',

    'stoch_k',
    'stoch_d',

    'macd',
    'macd_signal',
    'macd_diff',
]

In [6]:
COLS_TO_STD = FEATURES_COLS + TA_STD
COLS_TO_MIN_MAX = TA_MIN_MAX

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [8]:
df[COLS_TO_STD] = standard_scaler.fit_transform(df[COLS_TO_STD])
df[COLS_TO_MIN_MAX] = minmax_scaler.fit_transform(df[COLS_TO_MIN_MAX])

In [9]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,dom_sin,dom_cos,month,month_sin,month_cos,label,train_label,3b_label,3b_train_label,ttrain_label
count,123945.000000,123945.000000,123945.000000,123945.000000,1.239450e+05,1.239450e+05,1.239450e+05,123945.000000,123945.000000,1.239450e+05,...,1.239450e+05,123945.000000,123945.000000,1.239450e+05,1.239450e+05,123945.000000,123945.000000,123945.000000,123945.000000,123945.000000
mean,127.990416,128.035766,127.943652,127.990860,6.337400e+06,-1.797783e-16,1.429971e-15,0.000394,0.000003,-7.165908e-18,...,2.353235e-03,-0.022177,6.521973,-4.503608e-03,-4.841234e-03,0.017354,1.018484,0.023938,1.023938,1.017354
std,18.398600,18.410248,18.385790,18.398718,6.365523e+06,1.000004e+00,1.000004e+00,0.081064,0.000601,1.000004e+00,...,7.139405e-01,0.699857,3.439884,7.071395e-01,7.070488e-01,0.795467,0.823626,0.797676,0.797676,0.795467
min,101.349000,101.954000,101.182000,101.349000,0.000000e+00,-5.798381e-01,-1.423994e+01,-3.885500,-0.028329,-4.776263e+01,...,-9.987165e-01,-0.994869,1.000000,-1.000000e+00,-1.000000e+00,-1.000000,0.000000,-1.000000,0.000000,0.000000
25%,109.436000,109.460500,109.410500,109.435500,1.971010e+06,-3.795095e-01,-6.454883e-01,-0.027000,-0.000217,-3.651262e-01,...,-7.247928e-01,-0.758758,4.000000,-8.660254e-01,-8.660254e-01,-1.000000,0.000000,-1.000000,0.000000,0.000000
50%,130.790500,130.857000,130.725500,130.790500,4.233180e+06,-1.414836e-01,7.147442e-02,0.001000,0.000007,6.416603e-03,...,-2.449294e-16,-0.050649,7.000000,-2.449294e-16,-1.836970e-16,0.000000,1.000000,0.000000,1.000000,1.000000
75%,145.526000,145.580500,145.471000,145.527000,8.654850e+06,3.693064e-02,7.422525e-01,0.029500,0.000232,3.807668e-01,...,7.247928e-01,0.688967,10.000000,5.000000e-01,5.000000e-01,1.000000,2.000000,1.000000,2.000000,2.000000
max,161.941000,161.951000,161.917500,161.940000,2.393905e+08,2.958454e+01,3.856168e+00,2.052000,0.014463,2.385868e+01,...,9.987165e-01,1.000000,12.000000,1.000000e+00,1.000000e+00,1.000000,2.000000,1.000000,2.000000,2.000000


In [10]:
print(df[COLS_TO_STD].mean())         # Should be ~0
print(df[COLS_TO_STD].std(ddof=0))

close_log_return   -7.165908e-18
log_volume          1.429971e-15
spread             -1.797783e-16
ret_mean_5          4.356872e-18
ret_mean_10         1.238269e-17
ema_21             -1.445564e-15
sma_50             -5.503417e-17
atr_14              1.866576e-16
bb_upper            1.161221e-15
bb_lower           -1.456571e-15
bb_mavg             1.357510e-15
bb_width            1.801223e-16
donchian_upper     -7.337889e-17
donchian_lower     -1.052987e-15
donchian_mid       -2.531572e-16
stoch_k             2.678330e-16
stoch_d             1.016986e-16
macd               -9.630980e-18
macd_signal        -9.630980e-18
macd_diff           2.465072e-18
dtype: float64
close_log_return    1.0
log_volume          1.0
spread              1.0
ret_mean_5          1.0
ret_mean_10         1.0
ema_21              1.0
sma_50              1.0
atr_14              1.0
bb_upper            1.0
bb_lower            1.0
bb_mavg             1.0
bb_width            1.0
donchian_upper      1.0
donchian_lowe

In [11]:
import joblib
joblib.dump(standard_scaler, STD_SCALER_PATH)
joblib.dump(minmax_scaler, MINMAX_SCALER_PATH)
df.to_pickle(NORMALIZED_FILE_PATH)